In [1]:
import csv
import pandas as pd
import math
import requests

### Dataset Location
The Dataset cannot be pushed to Github. Github has decreased the allowed file size so the dataset will have to be downloaded manually.

The url is: https://data.cms.gov/provider-data/sites/default/files/resources/69a75aa9d3dc1aed6b881725cf0ddc12_1671768321/DAC_NationalDownloadableFile.csv

In [2]:
df = pd.read_csv('DAC_NationalDownloadableFile.csv', encoding='windows-1252', dtype='string')

In [3]:
# print(df.head())
addresses = df[['adr_ln_1', 'cty', 'st', 'zip','pri_spec']]
print(addresses.head())
print(addresses.shape[0])

               adr_ln_1               cty  st        zip  \
0  655 SAINT GEORGE AVE           ROSELLE  NJ  072032633   
1       2631 NW 41ST ST       GAINESVILLE  FL  326066689   
2        921 W 1ST N ST        MORRISTOWN  TN  378144549   
3        140 SYLVAN AVE  ENGLEWOOD CLIFFS  NJ  076322531   
4         57 CROTON AVE          OSSINING  NY  105624981   

                                         pri_spec  
0                                PHYSICAL THERAPY  
1                           CLINICAL PSYCHOLOGIST  
2                          CLINICAL SOCIAL WORKER  
3  REGISTERED DIETITIAN OR NUTRITION PROFESSIONAL  
4                                       OPTOMETRY  
2469709


In [4]:
batches = math.ceil(addresses.shape[0] / 10000)
print(batches)

247


### Steps to Get Geolocations of Addresses in Batch
1. Create a loop that generates a CSV for each 247 entries 
2. Send the generated CSV to the Census GOV Geolocation API for processing
3. Recieve the response and append it to a local CSV 
4. Rinse and repeat until all addresses are converted to geolocations (lat. and long.) to use with a map

In [5]:
URL = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
with open('test_addresses.csv', 'rb') as input_file:
    geo = requests.post(files={'addressFile': input_file}, url=URL, data={'benchmark':'Public_AR_Current'})

In [6]:
print(geo.content)

b'"000001","8 E View Dr, Whitehouse Station, NJ, 08889","Match","Exact","8 E VIEW DR, WHITEHOUSE STATION, NJ, 08889","-74.76707085499999,40.611750001000075","627779463","R"\n'
